# Unified Model with Integrated Bot: Highly Symmetric Case In-Depth

James Yu, 20 August 2022

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import inv
from sympy import *
np.set_printoptions(suppress=True) # disable scientific notation for readability

# 1. Verify the $x_{\infty}$ formula

This formula is

$$x_{\infty} = \frac{cd - \delta k_2}{cd + \delta k_1} z$$

In [2]:
def infinite_solution(A_n, A_c, delta, c, x_0, z):
    n = len(x_0)
    eps = np.finfo(np.float64).eps
    I = np.identity(n)
    O = np.zeros((n, n))
    
    Q = np.block([
        [I, O], 
        [O, O]
    ])
    A = np.block([
        [A_n, A_c], 
        [O, I]
    ])
    B = np.block([
        [I],
        [O]
    ])
    K_t = Q
    K_sequence = [K_t]
    
    while True: # generate solution matrices; note I divide delta into the inverse term for simplification
        K_t_new = Q + delta * A.T @ (K_t - K_t @ B @ inv(B.T @ K_t @ B + c * I / delta) @ B.T @ K_t) @ A
        K_sequence.insert(0, K_t_new)
        if np.allclose(K_t, K_t_new, rtol = eps, atol = eps): break
        K_t = K_t_new

    chi_var = np.block([
        [x_0], 
        [z]
    ])
    chi_ts = [chi_var]
    r_ts = []
    K_ss = K_sequence[0]
    L_ss = -inv(B.T @ K_ss @ B + c * I / delta) @ B.T @ K_ss @ A
    
    payoff = 0
    stage_payoffs = []
    discounted_stage_payoffs = []
    cumulative_payoffs = []
    
    i = 0
    while True:
        r_t = L_ss @ chi_var
        r_ts.append(r_t)
        p = -(chi_var.T @ Q @ chi_var + c * r_t.T @ r_t).item()
        payoff += delta**i * p
        stage_payoffs.append(p)
        discounted_stage_payoffs.append(delta**i * p)
        cumulative_payoffs.append(payoff)
        chi_var_new = A @ chi_var + B @ r_t
        chi_ts.append(chi_var_new)
        if np.allclose(chi_var, chi_var_new, rtol = eps, atol = eps): break
        chi_var = chi_var_new
        i += 1
        
    return chi_ts, K_ss #, r_ts, stage_payoffs, discounted_stage_payoffs, cumulative_payoffs, K_ss, L_ss

## Baseline

In [3]:
A_1_n = 0.99 * np.array([
    [0.5, 0.25, 0.25],
    [0.25, 0.5, 0.25],
    [0.25, 0.25, 0.5]
])
A_1_c = np.diag([0.01, 0.01, 0.01])
delta = 0.999
c = 1.0

x_0 = np.array([[10.0, 5.0, -20.0]], ndmin = 2).T
z = np.array([[5.0, 5.0, 5.0]], ndmin = 2).T

In [4]:
chi_ts, K_ss = infinite_solution(A_1_n, A_1_c, delta, c, x_0, z)

In [5]:
print(K_ss)

[[1.22185386 0.19077235 0.19077235 0.00423084 0.00280085 0.00280085]
 [0.19077235 1.22185386 0.19077235 0.00280085 0.00423084 0.00280085]
 [0.19077235 0.19077235 1.22185386 0.00280085 0.00280085 0.00423084]
 [0.00423084 0.00280085 0.00280085 0.0758686  0.01205966 0.01205966]
 [0.00280085 0.00423084 0.00280085 0.01205966 0.0758686  0.01205966]
 [0.00280085 0.00280085 0.00423084 0.01205966 0.01205966 0.0758686 ]]


In [6]:
K_ss[0:3, 0:3] # K_1

array([[1.22185386, 0.19077235, 0.19077235],
       [0.19077235, 1.22185386, 0.19077235],
       [0.19077235, 0.19077235, 1.22185386]])

In [7]:
K_ss[0:3, 3:6] # K_2

array([[0.00423084, 0.00280085, 0.00280085],
       [0.00280085, 0.00423084, 0.00280085],
       [0.00280085, 0.00280085, 0.00423084]])

In [8]:
k1 = sum(K_ss[0:3, 0:3])[0]
k2 = sum(K_ss[0:3, 3:6])[0]
k1 + k2

1.6132310912068586

In [9]:
sum(K_ss)

array([1.61323109, 1.61323109, 1.61323109, 0.10982046, 0.10982046,
       0.10982046])

In [10]:
d = A_1_c[0, 0]
z * ((c * d) - (delta * k2)) / ((c * d) + (delta * k1))

array([[0.00054999],
       [0.00054999],
       [0.00054999]])

In [11]:
chi_ts[-1] # x_ss

array([[0.00054999],
       [0.00054999],
       [0.00054999],
       [5.        ],
       [5.        ],
       [5.        ]])

They are the same.

## Different $x_0$

In [12]:
x_0_1 = np.array([[3.0, 4.0, 5.0]], ndmin = 2).T
chi_ts_1, K_ss_1 = infinite_solution(A_1_n, A_1_c, delta, c, x_0_1, z)
k1_1 = sum(K_ss_1[0:3, 0:3])[0]
k2_1 = sum(K_ss_1[0:3, 3:6])[0]
z * ((c * d) - (delta * k2_1)) / ((c * d) + (delta * k1_1))

array([[0.00054999],
       [0.00054999],
       [0.00054999]])

In [13]:
chi_ts_1[-1]

array([[0.00054999],
       [0.00054999],
       [0.00054999],
       [5.        ],
       [5.        ],
       [5.        ]])

## Lower $\delta$

In [14]:
delta_2 = 0.5
chi_ts_2, K_ss_2 = infinite_solution(A_1_n, A_1_c, delta_2, c, x_0, z)
k1_2 = sum(K_ss_2[0:3, 0:3])[0]
k2_2 = sum(K_ss_2[0:3, 3:6])[0]
z * ((c * d) - (delta_2 * k2_2)) / ((c * d) + (delta_2 * k1_2))

array([[0.04999505],
       [0.04999505],
       [0.04999505]])

In [15]:
chi_ts_2[-1]

array([[0.04999505],
       [0.04999505],
       [0.04999505],
       [5.        ],
       [5.        ],
       [5.        ]])

## Lower $c$

In [16]:
c_3 = 0.1
chi_ts_3, K_ss_3 = infinite_solution(A_1_n, A_1_c, delta, c_3, x_0, z)
k1_3 = sum(K_ss_3[0:3, 0:3])[0]
k2_3 = sum(K_ss_3[0:3, 3:6])[0]
z * ((c_3 * d) - (delta * k2_3)) / ((c_3 * d) + (delta * k1_3))

array([[0.000055],
       [0.000055],
       [0.000055]])

In [17]:
chi_ts_3[-1]

array([[0.000055],
       [0.000055],
       [0.000055],
       [5.      ],
       [5.      ],
       [5.      ]])

## Higher $c$

In [18]:
c_4 = 100.0
chi_ts_4, K_ss_4 = infinite_solution(A_1_n, A_1_c, delta, c_4, x_0, z)
k1_4 = sum(K_ss_4[0:3, 0:3])[0]
k2_4 = sum(K_ss_4[0:3, 3:6])[0]
z * ((c_4 * d) - (delta * k2_4)) / ((c_4 * d) + (delta * k1_4))

array([[0.05440648],
       [0.05440648],
       [0.05440648]])

In [19]:
chi_ts_4[-1]

array([[0.05440648],
       [0.05440648],
       [0.05440648],
       [5.        ],
       [5.        ],
       [5.        ]])

## Different $z$

In [20]:
z_5 = np.array([[20.0, 20.0, 20.0]], ndmin = 2).T
chi_ts_5, K_ss_5 = infinite_solution(A_1_n, A_1_c, delta, c, x_0, z_5)
k1_5 = sum(K_ss_5[0:3, 0:3])[0]
k2_5 = sum(K_ss_5[0:3, 3:6])[0]
z_5 * ((c * d) - (delta * k2_5)) / ((c * d) + (delta * k1_5))

array([[0.00219996],
       [0.00219996],
       [0.00219996]])

In [21]:
chi_ts_5[-1]

array([[ 0.00219996],
       [ 0.00219996],
       [ 0.00219996],
       [20.        ],
       [20.        ],
       [20.        ]])

## Different network

In [22]:
A_6_n = 0.5 * np.array([
    [0.4, 0.3, 0.3],
    [0.3, 0.4, 0.3],
    [0.3, 0.3, 0.4]
])
A_6_c = np.diag([0.5, 0.5, 0.5])
d_6 = A_6_c[0, 0]
chi_ts_6, K_ss_6 = infinite_solution(A_6_n, A_6_c, delta, c, x_0, z)
k1_6 = sum(K_ss_6[0:3, 0:3])[0]
k2_6 = sum(K_ss_6[0:3, 3:6])[0]
z * ((c * d_6) - (delta * k2_6)) / ((c * d_6) + (delta * k1_6))

array([[1.00160096],
       [1.00160096],
       [1.00160096]])

In [23]:
chi_ts_6[-1]

array([[1.00160096],
       [1.00160096],
       [1.00160096],
       [5.        ],
       [5.        ],
       [5.        ]])

# 2. Symbolic expressions for $k_1$,  $k_2$ and $x_{\infty}$

In [24]:
x, d, z, r, delta, c, k1, k2, k3 = symbols("x d z r delta c k_1 k_2 k_3")
xprime = (1 - d) * x + d * z + r # the next period function
xprime

d*z + r + x*(1 - d)

In [25]:
v_x = -(x**2) - (c * r**2) - (delta * k1 * xprime**2) - (2 * delta * k2 * xprime * z) - (delta * k3)
v_x

-c*r**2 - delta*k_1*(d*z + r + x*(1 - d))**2 - 2*delta*k_2*z*(d*z + r + x*(1 - d)) - delta*k_3 - x**2

In [26]:
diff(v_x, r)

-2*c*r - delta*k_1*(2*d*z + 2*r + 2*x*(1 - d)) - 2*delta*k_2*z

In [27]:
expand(diff(v_x, r))

-2*c*r + 2*d*delta*k_1*x - 2*d*delta*k_1*z - 2*delta*k_1*r - 2*delta*k_1*x - 2*delta*k_2*z

In [28]:
# the FOC from the notes, multiplied on both sides by -2
expand(-2 * (delta * (1 - d) * x * k1 + delta * d * z * k1 + delta * k2 * z + r * (delta * k1 + c)))

-2*c*r + 2*d*delta*k_1*x - 2*d*delta*k_1*z - 2*delta*k_1*r - 2*delta*k_1*x - 2*delta*k_2*z

In [29]:
solve(diff(v_x, r), r)

[delta*(d*k_1*x - d*k_1*z - k_1*x - k_2*z)/(c + delta*k_1)]

In [30]:
r_star = solve(diff(v_x, r), r)[0] # only one solution from above
r_star

delta*(d*k_1*x - d*k_1*z - k_1*x - k_2*z)/(c + delta*k_1)

In [31]:
simplify(expand(r_star))

delta*(d*k_1*x - d*k_1*z - k_1*x - k_2*z)/(c + delta*k_1)

We can take this opportunity to solve for $x_{\infty}$:

$$x_{\infty} = (1 - d)x_{\infty} + dz + r_{ss}$$

In [32]:
solve((1-d)*x + d*z + r_star - x, x)[0]

z*(c*d - delta*k_2)/(c*d + delta*k_1)

Back to the value function:

In [33]:
simplify(expand(v_x).subs(r, r_star))

(-c*d**2*delta*k_1*x**2 + 2*c*d**2*delta*k_1*x*z - c*d**2*delta*k_1*z**2 + 2*c*d*delta*k_1*x**2 - 2*c*d*delta*k_1*x*z + 2*c*d*delta*k_2*x*z - 2*c*d*delta*k_2*z**2 - c*delta*k_1*x**2 - 2*c*delta*k_2*x*z - c*delta*k_3 - c*x**2 - delta**2*k_1*k_3 + delta**2*k_2**2*z**2 - delta*k_1*x**2)/(c + delta*k_1)

First, we need terms containing $x^2$. This gives the following expression for $k_1$:

In [34]:
k1_solved = simplify(-(-c*d**2 * delta * k1 + 2*c*d*delta*k1 - c*delta*k1 - c - delta*k1)/(c+delta*k1))
k1_solved

(c*d**2*delta*k_1 - 2*c*d*delta*k_1 + c*delta*k_1 + c + delta*k_1)/(c + delta*k_1)

which we see simplifies to $k_1 = 1 + \frac{c\delta k_1(1-d)^2}{c+\delta k_1}$ as in the note.

In [35]:
k1_solved_simple = 1 + (c*delta*k1*(1-d)**2)/(c+delta*k1)
k1_solved_simple

c*delta*k_1*(1 - d)**2/(c + delta*k_1) + 1

In [36]:
simplify(expand((k1_solved_simple - k1) * (c + delta * k1))) # multiply both sides by the denominator

c*d**2*delta*k_1 - 2*c*d*delta*k_1 + c*delta*k_1 - c*k_1 + c - delta*k_1**2 + delta*k_1

Then $-\delta k_1^2 + k_1 (\delta - c + c\delta(1-d)^2) + c = 0$

In [37]:
g = symbols("g")
solve(-delta*k1**2 + k1*g + c, k1)

[(g - sqrt(4*c*delta + g**2))/(2*delta),
 (g + sqrt(4*c*delta + g**2))/(2*delta)]

In [38]:
solve(-delta*k1**2 + k1*g + c, k1)[0].subs(g, delta-c+c*delta*(1-d)**2)

(c*delta*(1 - d)**2 - c + delta - sqrt(4*c*delta + (c*delta*(1 - d)**2 - c + delta)**2))/(2*delta)

In [39]:
solve(-delta*k1**2 + k1*g + c, k1)[0].subs(g, delta-c+c*delta*(1-d)**2).subs(delta, 0.9).subs(d, 0.5).subs(c, 1)

-0.986933159433690

In [40]:
1 + (1*0.9*(1-0.5)**2 * -0.98693315943369) / (1 + 0.9 * -0.98693315943369)

-0.9869331594336914

In [41]:
solve(-delta*k1**2 + k1*g + c, k1)[1].subs(g, delta-c+c*delta*(1-d)**2)

(c*delta*(1 - d)**2 - c + delta + sqrt(4*c*delta + (c*delta*(1 - d)**2 - c + delta)**2))/(2*delta)

In [42]:
solve(-delta*k1**2 + k1*g + c, k1)[1].subs(g, delta-c+c*delta*(1-d)**2).subs(delta, 0.9).subs(d, 0.5).subs(c, 1)

1.12582204832258

In [43]:
_, K_test = infinite_solution(A_6_n, A_6_c, 0.9, 1, x_0, np.array([[5.0, 5.0, 5.0]], ndmin = 2).T)
print(K_test)

[[1.04273065 0.0415457  0.0415457  0.06210507 0.04996829 0.04996829]
 [0.0415457  1.04273065 0.0415457  0.04996829 0.06210507 0.04996829]
 [0.0415457  0.0415457  1.04273065 0.04996829 0.04996829 0.06210507]
 [0.06210507 0.04996829 0.04996829 1.45350884 0.21173007 0.21173007]
 [0.04996829 0.06210507 0.04996829 0.21173007 1.45350884 0.21173007]
 [0.04996829 0.04996829 0.06210507 0.21173007 0.21173007 1.45350884]]


In [44]:
sum(K_test[0:3, 0:3])

array([1.12582205, 1.12582205, 1.12582205])

The positive (second) solution is the one which occurs in practice.

Back to the value function again:

In [45]:
simplify(expand(v_x).subs(r, r_star))

(-c*d**2*delta*k_1*x**2 + 2*c*d**2*delta*k_1*x*z - c*d**2*delta*k_1*z**2 + 2*c*d*delta*k_1*x**2 - 2*c*d*delta*k_1*x*z + 2*c*d*delta*k_2*x*z - 2*c*d*delta*k_2*z**2 - c*delta*k_1*x**2 - 2*c*delta*k_2*x*z - c*delta*k_3 - c*x**2 - delta**2*k_1*k_3 + delta**2*k_2**2*z**2 - delta*k_1*x**2)/(c + delta*k_1)

Taking terms with $x$, we get the following expression for $k_2$:

In [46]:
simplify(-0.5 * (2*c*d**2*delta*k1 - 2*c*d*delta*k1 + 2*c*d*delta*k2 - 2*c*delta*k2) / (c+delta*k1))

1.0*c*delta*(-d**2*k_1 + d*k_1 - d*k_2 + k_2)/(c + delta*k_1)

This evaluates to $k_2 = \frac{c\delta (1-d)(dk_1 + k_2)}{c+\delta k_1}$.

In [47]:
solve(simplify((c*delta*(1-d)*(d*k1+k2))/(c+delta*k1) - k2), k2)[0]

c*d*delta*k_1*(1 - d)/(c*d*delta - c*delta + c + delta*k_1)